In [1]:
%cd ..

/home/prayson/Code/hadithi/dev.io


In [172]:
from pathlib import Path
from zipfile import ZipFile

from httpx import AsyncClient, Client
from mimesis import Person
from mimesis.locales import Locale
import jmespath
import pandas as pd

In [89]:
MIMESIS_LOCALE_URL = "https://mimesis.name/en/master/locales.html#overriding-locale"
COUNTRY_CODE_CSV = "assets/languages.csv"
[language_codes, *_] = pd.read_html(MIMESIS_LOCALE_URL)
country_codes = pd.read_csv(COUNTRY_CODE_CSV, usecols=["Country", "Two Letter"])

In [110]:
country_iso = (
    country_codes
     .transform(lambda d: d.str.lower())
     .rename(columns={"Country": "country", "Two Letter":"code"})
     #.drop_duplicates(subset=["country"], keep="first")
     .assign(code = lambda d: d["code"].str.split("-").str[0])
     .query("code != 'en'")
)

In [111]:
country_language = (language_codes[["Code", "Name"]]
     .transform(lambda d: d.str.lower())
     .merge(country_iso, how="left", left_on="Code", right_on="code")
     .dropna(subset=["country", "code"])
     .loc[:, ["code", "country"]]
     .set_index("country")
     .to_dict()
     .get("code")

)

In [5]:
DATA_DIR = Path("assets/books.zip")

with ZipFile(DATA_DIR) as z:
    print(z.namelist())
    files = (file for file in z.namelist() if Path(file).suffix == ".csv")
    data = {Path(file).stem.lower(): pd.read_csv(z.extract(file),low_memory=False) for file in files}     

['Books.csv', 'Ratings.csv', 'Users.csv', 'recsys_taxonomy2.png']


In [140]:

def get_name(language:str) -> str:
    person = Person(language)
    return person.full_name()


In [156]:
users = pd.read_csv("assets/users.csv", dtype_backend="pyarrow", dtype={"age":"Int64"})

In [157]:
users

,user_id,fake_name,location,age
0,1,Catrice Riggs,"nyc, new york, usa",<NA>
1,2,Johnson Colon,"stockton, california, usa",18
2,3,Лолита Кузьмин,"moscow, yukon territory, russia",<NA>
3,4,Abraim Melo,"porto, v.n.gaia, portugal",17
4,5,Barton King,"farnborough, hants, united kingdom",<NA>
...,...,...,...,...
278853,278854,Renaldo Gomez,"portland, oregon, usa",<NA>
278854,278855,Elizbeth Bird,"tacoma, washington, united kingdom",50
278855,278856,Arda Guichard,"brampton, ontario, canada",<NA>
278856,278857,Clifford Erickson,"knoxville, tennessee, usa",<NA>


In [221]:
client = Client(base_url="https://openlibrary.org/api")
ISBN = "0140301690"
r = client.get(url="/books", params={"bibkeys": f"ISBN:{ISBN}", "format":"json"}, timeout=10)

In [222]:
b = r.json()
b

{'ISBN:0140301690': {'bib_key': 'ISBN:0140301690',
  'info_url': "https://openlibrary.org/books/OL26336110M/Alice's_Adventures_in_Wonderland",
  'preview': 'borrow',
  'preview_url': 'https://archive.org/details/isbn_0140301690',
  'thumbnail_url': 'https://covers.openlibrary.org/b/id/8596023-S.jpg'}}

In [223]:
open_client = Client(base_url="https://openlibrary.org")

In [224]:
[url, *_] = jmespath.search("*.info_url", b)
work = url.split("/")[-2]

r = open_client.get(f"/books/{work}.json", timeout=10)
r

<Response [200 OK]>

In [225]:
r.json()

{'notes': '"Text of this edition of Alice in Wonderland is that of the first edition."',
 'identifiers': {'google': ['9hOE1JPD3'], 'goodreads': ['11491198']},
 'title': "Alice's Adventures in Wonderland",
 'subtitle': 'and Through the Looking-Glass',
 'authors': [{'key': '/authors/OL22098A'}],
 'publish_date': '/1970',
 'other_titles': ['Through the looking-glass.'],
 'publishers': ['Puffin Books'],
 'covers': [8596023],
 'physical_format': 'Paperback',
 'full_title': "Alice's adventures in Wonderland and Through the looking glass",
 'publish_places': ['London, England'],
 'contributions': ['Tenniel, John, 1820-1914, illustrator'],
 'pagination': '345 pages :',
 'source_records': ['ia:isbn_0140301690', 'bwb:9780140301694'],
 'work_titles': ["Alice's adventures in Wonderland"],
 'languages': [{'key': '/languages/eng'}],
 'subjects': ['Fantasy'],
 'publish_country': 'enk',
 'by_statement': 'by Lewis Carroll ; with ninety-two illustrations by John Tenniel',
 'type': {'key': '/type/edition

In [226]:
b = r.json()

g = open_client.get(url=f'{b.get("works")[0].get("key")}.json', timeout=10)
g

<Response [200 OK]>

In [227]:
g.json()

{'title': "Alice's Adventures in Wonderland / Through the Looking Glass",
 'key': '/works/OL151411W',
 'authors': [{'author': {'key': '/authors/OL22098A'},
   'type': {'key': '/type/author_role'}}],
 'type': {'key': '/type/work'},
 'description': "A very real little girl named Alice follows a remarkable rabbit down a rabbit hole and steps through a looking-glass to come face to face with some of the strangest adventures and some of the oddest characters in all literature. The crusty Duchess, the Mad Hatter, the weeping Mock Turtle, the diabolical Queen of Hearts, the Cheshire-Cat, Tweedledum and Tweedledee--each one is more eccentric, and more entertaining, than the last. And all of them could only have come from the pen of Lewis Carroll, one of the few adults ever to enter successfully the children's world of make-believe--a wonderland where the impossible becomes possible, the unreal, real...where the heights of adventure are limited only by the depths of imagination.\r\n--back cover

In [158]:
# https://openlibrary.org/works/OL18020194W/bookshelves.json
# https://openlibrary.org/works/OL18020194W/ratings.json

In [220]:
data["books"].sort_values("Year-Of-Publication")

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
212233,0947901051,Collect Post Office Cards,A. Roberts,0,Benham (A. Buckingham) Ltd,http://images.amazon.com/images/P/0947901051.0...,http://images.amazon.com/images/P/0947901051.0...,http://images.amazon.com/images/P/0947901051.0...
51621,0006479561,Sacred Clowns,Tony Hillerman,0,Fairmount Books Ltd Remainders,http://images.amazon.com/images/P/0006479561.0...,http://images.amazon.com/images/P/0006479561.0...,http://images.amazon.com/images/P/0006479561.0...
193438,0140382690,The Enemy You Killed,Peter McFarlane,0,Penguin Books Australia Ltd,http://images.amazon.com/images/P/0140382690.0...,http://images.amazon.com/images/P/0140382690.0...,http://images.amazon.com/images/P/0140382690.0...
25016,0879972734,Walkers On the Sky,David J Lake,0,Penguin Putnam~mass,http://images.amazon.com/images/P/0879972734.0...,http://images.amazon.com/images/P/0879972734.0...,http://images.amazon.com/images/P/0879972734.0...
17668,0590023888,Misty of Chincoteague,Marguerite Henry,0,Scholastic Inc,http://images.amazon.com/images/P/0590023888.0...,http://images.amazon.com/images/P/0590023888.0...,http://images.amazon.com/images/P/0590023888.0...
...,...,...,...,...,...,...,...,...
80264,0140301690,Alice's Adventures in Wonderland and Through t...,Lewis Carroll,2050,Puffin Books,http://images.amazon.com/images/P/0140301690.0...,http://images.amazon.com/images/P/0140301690.0...,http://images.amazon.com/images/P/0140301690.0...
97826,0140201092,Outline of European Architecture (Pelican S.),Nikolaus Pevsner,2050,Penguin USA,http://images.amazon.com/images/P/0140201092.0...,http://images.amazon.com/images/P/0140201092.0...,http://images.amazon.com/images/P/0140201092.0...
209538,078946697X,"DK Readers: Creating the X-Men, How It All Beg...",2000,DK Publishing Inc,http://images.amazon.com/images/P/078946697X.0...,http://images.amazon.com/images/P/078946697X.0...,http://images.amazon.com/images/P/078946697X.0...,NaN
221678,0789466953,"DK Readers: Creating the X-Men, How Comic Book...",2000,DK Publishing Inc,http://images.amazon.com/images/P/0789466953.0...,http://images.amazon.com/images/P/0789466953.0...,http://images.amazon.com/images/P/0789466953.0...,NaN
